# Evaluación de PC-SMOTE con Grid Search en el dataset Shuttle (Generación de caso base y datasets aumentados)


In [1]:
# lo que hace es modificar la lista de rutas de búsqueda de módulos de Python (sys.path) para incluir las carpetas ../scripts y ../datasets como ubicaciones adicionales donde Python puede buscar módulos o paquetes cuando hacés un import.
import sys
sys.path.append("../scripts")
sys.path.append("../datasets")

## Importación de módulos y librerías necesarias


In [2]:
# --- Módulos propios del proyecto ---
from cargar_dataset import cargar_dataset                      # Función para cargar datasets según configuración
from config_datasets import config_datasets                    # Diccionario de configuración de datasets
from evaluacion import evaluar_sampler_holdout                 # Evaluación de sobremuestreo con partición hold-out
from custom_samplers import PCSMOTEWrapper                     # Wrapper personalizado para la técnica PCSMOTE
from pc_smote import PCSMOTE                                   # Implementación principal de PCSMOTE

# --- Librerías estándar de Python ---
from datetime import datetime, timedelta                       # Manejo de fechas y tiempos
from itertools import product                                  # Generación de combinaciones de parámetros
import os                                                      # Operaciones con el sistema de archivos

# --- Librerías científicas ---
import numpy as np                                              # Operaciones numéricas y algebra lineal
import pandas as pd                                             # Manipulación y análisis de datos tabulares
from scipy.stats import uniform                                 # Distribuciones para búsqueda de hiperparámetros

# --- Scikit-learn: preprocesamiento ---
from sklearn.preprocessing import LabelEncoder, StandardScaler # Codificación de etiquetas y escalado de datos
from sklearn.pipeline import make_pipeline, Pipeline            # Creación de pipelines de procesamiento y modelado

# --- Scikit-learn: división y validación ---
from sklearn.model_selection import (
    train_test_split,                                           # División de datos en train/test
    StratifiedKFold,                                            # Validación cruzada estratificada
    RandomizedSearchCV                                          # Búsqueda aleatoria de hiperparámetros
)

# --- Scikit-learn: reducción de dimensionalidad ---
from sklearn.decomposition import PCA                           # Análisis de Componentes Principales

# --- Scikit-learn: métricas ---
from sklearn.metrics import (
    f1_score,                                                    # Métrica F1-Score
    balanced_accuracy_score,                                     # Precisión balanceada
    matthews_corrcoef,                                           # Coeficiente MCC
    cohen_kappa_score,                                           # Kappa de Cohen
    make_scorer                                            
)

# --- Scikit-learn: clasificadores ---
from sklearn.ensemble import RandomForestClassifier             # Clasificador Random Forest
from sklearn.linear_model import LogisticRegression             # Regresión logística
from sklearn.svm import SVC                                      # Máquinas de Vectores de Soporte (SVM)

from sklearn.exceptions import ConvergenceWarning
import warnings

### Evaluación de modelos con validación cruzada estratificada

Para evaluar el rendimiento de los modelos de clasificación sobre los datasets previamente balanceados, se utilizó validación cruzada estratificada de 5 particiones (Stratified K-Fold con *k=5*). Este método garantiza que en cada fold de entrenamiento y validación se preserve la proporción original de clases, lo cual es especialmente importante en tareas de clasificación multiclase con datasets balanceados artificialmente.

Durante el proceso, cada modelo es entrenado y evaluado cinco veces, cada vez usando un subconjunto distinto como conjunto de prueba y el resto como conjunto de entrenamiento. Las métricas calculadas en cada iteración (F1-score macro, balanced accuracy, MCC y kappa de Cohen) se promedian para obtener un valor representativo y del rendimiento general del modelo sobre ese dataset aumentado.

Este enfoque evita sobreajuste y proporciona una evaluación más confiable que una simple división train/test, permitiendo comparar de forma justa distintas configuraciones de sobremuestreo y modelos de clasificación.


In [ ]:
warnings.filterwarnings("ignore", category=ConvergenceWarning)

# --- Definición de modelos e hiperparámetros (base) ---
modelos = {
    "SVM": {
        "pipeline": Pipeline([('classifier', SVC(random_state=42))]),
        "param_distributions": {
            'classifier__C': uniform(0.1, 10),
            'classifier__kernel': ['linear', 'rbf'],
            'classifier__gamma': ['scale', 'auto']
        }
    },
    "LogisticRegression": {
        "pipeline": Pipeline([('classifier', LogisticRegression(max_iter=1000, random_state=42))]),
        "param_distributions": {
            'classifier__C': uniform(0.1, 10),
            'classifier__penalty': ['l2'],
            'classifier__solver': ['lbfgs']
        }
    },
    "RandomForest": {
        "pipeline": Pipeline([('classifier', RandomForestClassifier(random_state=42))]),
        "param_distributions": {
            'classifier__n_estimators': [100, 200, 300],
            'classifier__max_depth': [None, 10, 20],
            'classifier__min_samples_split': [2, 5, 10]
        }
    }
}

# --- Rutas ---
ruta_aug  = "../datasets/datasets_aumentados/"
ruta_base = "../datasets/datasets_aumentados/base/"

# --- Armar pares (train/test) tanto en aumentados como en base ---
def listar_pares(ruta, tipo):
    pares = []
    if not os.path.isdir(ruta):
        return pares
    for f in os.listdir(ruta):
        if not (f.endswith("_train.csv") and os.path.isfile(os.path.join(ruta, f))):
            continue
        path_train = os.path.join(ruta, f)
        path_test  = os.path.join(ruta, f.replace("_train.csv", "_test.csv"))
        if not os.path.isfile(path_test):
            print(f"⚠️ Falta el test para: {f} -> esperado: {os.path.basename(path_test)}")
            continue
        pares.append({
            "tipo": tipo,
            "nombre_train": f,
            "path_train": path_train,
            "path_test": path_test
        })
    return pares

pares = listar_pares(ruta_aug, "aumentado") + listar_pares(ruta_base, "base")

# Orden estable para reproducibilidad (por tipo y nombre de train)
pares = sorted(pares, key=lambda x: (x["tipo"], x["nombre_train"]))

# --- Métricas personalizadas para CV ---
scoring = {
    'f1_macro': 'f1_macro',
    'balanced_accuracy': 'balanced_accuracy',
    'mcc': make_scorer(matthews_corrcoef),
    'cohen_kappa': make_scorer(cohen_kappa_score)
}

# --- Acumuladores de resultados (un CSV por modelo con todo junto) ---
resultados_por_modelo = {nombre: [] for nombre in modelos}

# --- Contar total de pares ---
total_pares = len(pares)
contador_par = 0

# --- Helper: carga CSV con fallback de target ---
def cargar_xy(path_csv):
    df = pd.read_csv(path_csv)
    if "target" in df.columns:
        X = df.drop(columns=["target"]).values
        y = df["target"].values
    else:
        X = df.iloc[:, :-1].values
        y = df.iloc[:, -1].values
    return X, y

# --- Evaluación por par (train/test) ---
for item in pares:
    contador_par += 1
    archivo_train = item["nombre_train"]
    ruta_train = item["path_train"]
    ruta_test  = item["path_test"]
    tipo = item["tipo"]

    # Parseo de metadatos desde el nombre *_train.csv
    nombre = archivo_train.replace(".csv", "")
    partes = nombre.split("_")
    if tipo == "aumentado":
        # Formato esperado: pcsmote_<dataset>_D<densidad>_R<riesgo>_P<pureza>_train
        if len(partes) < 6:
            print(f"⚠️ Nombre inválido/incompleto (aumentado): {archivo_train}")
            continue
        tecnica = partes[0]
        nombre_dataset = partes[1]
        densidad = partes[2][1:]  # quita 'D'
        riesgo   = partes[3][1:]  # quita 'R'
        pureza   = partes[4][1:]  # quita 'P'
    else:
        # Base: <dataset>_train
        tecnica = "base"
        nombre_dataset = nombre.replace("_train", "")
        densidad = "NA"
        riesgo   = "NA"
        pureza   = "NA"

    # Progreso
    print(f"\n📂 ({contador_par}/{total_pares}) Par: {archivo_train}  (tipo: {tipo})")
    print(f"🔎 Técnica: {tecnica} | Dataset: {nombre_dataset} | Densidad: {densidad} | Riesgo: {riesgo} | Pureza: {pureza}")

    # Carga train/test
    try:
        X_train, y_train = cargar_xy(ruta_train)
        X_test,  y_test  = cargar_xy(ruta_test)
    except Exception as e:
        print(f"❌ Error al leer train/test ({archivo_train}): {e}")
        continue

    # Heurística: datasets grandes (ej. shuttle) => simplificar SVM
    n_samples = X_train.shape[0]
    es_grande = (n_samples >= 30000) or (nombre_dataset.lower() == "shuttle")

    # CV base y grande
    cv_base = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    cv_grande = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

    # Búsqueda aleatoria + CV por modelo
    for nombre_modelo, info in modelos.items():
        print(f"⚙️ Validando modelo con Pipeline y RandomizedSearchCV: {nombre_modelo}")

        # Config local por modelo (sin mutar el dict base)
        pipeline_local = None
        params_local = None
        cv_actual = cv_base
        n_iter_actual = 10
        n_jobs_actual = -1

        if nombre_modelo == "SVM":
            pipeline_local = Pipeline([
                ('scaler', StandardScaler()),
                ('classifier', SVC(random_state=42))
            ])
            if es_grande:
                params_local = {
                    'classifier__kernel': ['linear'], # agregar mas kernel (gaussiano, radiales, cubicos, etc)
                    'classifier__C': uniform(0.1, 10)
                }
                pipeline_local.named_steps['classifier'].set_params(max_iter=5000, cache_size=1000)
                cv_actual = cv_grande
                n_iter_actual = 5
                n_jobs_actual = 1
            else:
                params_local = info['param_distributions']

        elif nombre_modelo == "LogisticRegression":
            pipeline_local = Pipeline([
                ('scaler', StandardScaler()),
                ('classifier', LogisticRegression(max_iter=1000, random_state=42))
            ])
            params_local = info['param_distributions']

        else:  # RandomForest
            pipeline_local = info['pipeline']
            params_local = info['param_distributions']
            # agregar mas parametros para ajustar. Buscar hipercalibration
            
            # aumentar info de la pcsmote en los archivos de log

        try:
            search = RandomizedSearchCV(
                estimator=pipeline_local,
                param_distributions=params_local,
                n_iter=n_iter_actual,
                cv=cv_actual,
                scoring=scoring,
                refit='f1_macro',      # refitea el mejor en TODO el train
                random_state=42,
                n_jobs=n_jobs_actual,
                pre_dispatch="2*n_jobs",
                verbose=2,
                error_score='raise'
            )
            search.fit(X_train, y_train)

            # Métricas CV en el mejor índice
            idx = search.best_index_
            cv_f1   = search.cv_results_['mean_test_f1_macro'][idx]
            cv_bacc = search.cv_results_['mean_test_balanced_accuracy'][idx]
            cv_mcc  = search.cv_results_['mean_test_mcc'][idx]
            cv_kappa= search.cv_results_['mean_test_cohen_kappa'][idx]

            # Evaluación out-of-sample en TEST con el mejor estimador ya refiteado
            best_est = search.best_estimator_
            y_pred = best_est.predict(X_test)

            test_f1   = f1_score(y_test, y_pred, average='macro')
            test_bacc = balanced_accuracy_score(y_test, y_pred)
            test_mcc  = matthews_corrcoef(y_test, y_pred)
            test_kappa= cohen_kappa_score(y_test, y_pred)

            resultados_por_modelo[nombre_modelo].append({
                # metadata
                'dataset': nombre_dataset,
                'tecnica': tecnica,
                'densidad': densidad,
                'riesgo': riesgo,
                'pureza': pureza,
                'modelo': nombre_modelo,
                'mejor_configuracion': str(search.best_params_),
                # métricas CV (train CV)
                'cv_f1_macro': cv_f1,
                'cv_balanced_accuracy': cv_bacc,
                'cv_mcc': cv_mcc,
                'cv_cohen_kappa': cv_kappa,
                # métricas TEST (out-of-sample)
                'test_f1_macro': test_f1,
                'test_balanced_accuracy': test_bacc,
                'test_mcc': test_mcc,
                'test_cohen_kappa': test_kappa,
            })

            # Guardado parcial (sobrescribe) por modelo
            os.makedirs("../resultados", exist_ok=True)
            output_path = f"../resultados/resultados_{nombre_modelo}.csv"
            pd.DataFrame(resultados_por_modelo[nombre_modelo]).to_csv(output_path, index=False)

        except Exception as e:
            print(f"❌ Error al validar {nombre_modelo} en {archivo_train}: {e}")

# --- Persistencia final (por si quedó algo sin volcar) ---
os.makedirs("../resultados", exist_ok=True)
for nombre_modelo, lista_resultados in resultados_por_modelo.items():
    df_final = pd.DataFrame(lista_resultados)
    output_path = f"../resultados/resultados_{nombre_modelo}.csv"
    df_final.to_csv(output_path, index=False)
    print(f"📁 Resultados guardados: {output_path}")
